In [1]:
import numpy as np 
import cv2 
from matplotlib import pyplot as plt 

import tensorflow as tf 
from tensorflow.keras import models, layers

import os
from tqdm import tqdm, trange
from pynput.keyboard import Key, Controller
Keys = Controller()

from Segmentor_App import segmentor

In [2]:
def press(key):
    Keys.press(key)
    Keys.release(key)

In [3]:
def assignFlags():
    flagDict = {
        "flagMotion": False,    
        "flagSkin": False,
        "flagContour": False,
    }
    return flagDict 


In [4]:
hand_model = models.load_model('HandModel')
mask_model = models.load_model('MaskModel')

In [5]:
dir_list = ['frame', 'motionMask', 'motion', 'skinMask', 'skin', 'finalMask', 'finalHand']

cap = cv2.VideoCapture(0)

height = int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT))
width = int(cap.get(cv2.CAP_PROP_FRAME_WIDTH))
    
boundingBox = {'cx': width//2, 'cy': height//2}
back = np.zeros((height, width, 3))

returnDict = {}                 
flags = assignFlags()

segment = segmentor()
bgSubthreshold = 20
bgSubtractorLr = 0

boxWidth = 256
boxHeight = 256

flagDetect = False
frame_rate = 20
curr_frame = 1

while cap.isOpened():

    _, frame = cap.read()
    box = frame.copy()
    blur = cv2.GaussianBlur(frame, (5, 5), cv2.BORDER_DEFAULT)
    drawing = np.zeros((height, width, 3), dtype = np.uint8)
    finalMask = np.zeros((height, width), dtype = np.uint8)

    startCoord = (max(0, boundingBox["cx"] - boxWidth//2), max(0, boundingBox["cy"] - boxHeight//2))
    endCoord = (min(width, boundingBox["cx"] + boxWidth//2), min(height, boundingBox["cy"] + boxHeight//2))

    cv2.rectangle(box, startCoord, endCoord, 0xFF0000, 1)

    returnDict["blur"] = blur
    returnDict["frame"] = frame
    returnDict["drawing"] = drawing
    returnDict["finalMask"] = finalMask

    cv2.imshow('Frame', frame)
    cv2.imshow('Blurred', blur)
    cv2.imshow('Box', box)
        
    key = cv2.waitKey(1) & 0xFF

    if key == ord('b'):

        bgSubtractor = cv2.createBackgroundSubtractorMOG2(10, bgSubthreshold)
        back = frame
        keys = flags.keys()
        for i in keys:
            flags[i] = True

    if flags["flagMotion"] == True:
        returnDict = segment.detect_motion(bgSubtractor, bgSubtractorLr, returnDict)
        cv2.imshow('Motion Mask', returnDict["motionMask"])
        cv2.imshow('Motion', returnDict["motion"])

    if flags["flagSkin"] == True:
        returnDict = segment.detect_skin(returnDict)
        cv2.imshow('Skin Mask', returnDict["skinMask"])
        cv2.imshow('Skin', returnDict["skin"])

    if flags["flagContour"] == True:
        returnDict, boundingBox = segment.get_contour(returnDict, boundingBox)
        cv2.imshow('Final Mask', returnDict["finalMask"])
        if "finalHand" in returnDict.keys():
            cv2.imshow('Final Hand', returnDict["finalHand"])
        cv2.imshow('Drawing', returnDict["drawing"])

    if flagDetect == True:
        
        curr_frame += 1
        hand = returnDict["finalHand"][startCoord[1]:endCoord[1], startCoord[0]:endCoord[0], :]
        mask = returnDict["finalMask"][startCoord[1]:endCoord[1], startCoord[0]:endCoord[0]]
        hand = np.expand_dims(hand, 0)
        mask = np.expand_dims(mask, 0)
        mask = np.expand_dims(mask, -1)
        
        if(curr_frame == frame_rate):
            try:
                p1 = hand_model.predict(hand)
                p2 = mask_model.predict(mask)
                
                p1_idx = np.argmax(p1)
                p2_idx = np.argmax(p2)

                p = p1_idx if p1[0][p1_idx] >= p2[0][p2_idx] else p2_idx 

                if p == 3:
                    print('rotate')
                    press(Key.up)
                elif p == 2:
                    print('left')
                    press(Key.left)
                elif p == 1:
                    print('right')
                    press(Key.right)
                elif p == 0:
                    print('down')
                    press(Key.down)
                print('Done')

                # print(p)

            except: print('No gesture detected')
            curr_frame = 1
    
    if key == ord('p'):
            #     print('rotate')
            # elif p == 2:
            #     print('left
        flagDetect = True
    
    if key == ord('s'):
        flagDetect = False 

    if key == ord('q'):
        break

cap.release()
cv2.destroyAllWindows()

rotate
Done
rotate
Done
No gesture detected
rotate
Done
rotate
Done
rotate
Done
rotate
Done
rotate
Done
rotate
Done
down
Done
rotate
Done
rotate
Done
rotate
Done
rotate
Done
rotate
Done
rotate
Done
rotate
Done
rotate
Done
No gesture detected
No gesture detected
down
Done
down
Done
rotate
Done
rotate
Done
rotate
Done
rotate
Done
rotate
Done
rotate
Done
rotate
Done
rotate
Done
rotate
Done
rotate
Done
rotate
Done
rotate
Done
rotate
Done
rotate
Done
rotate
Done
rotate
Done
No gesture detected
rotate
Done
down
Done
rotate
Done
rotate
Done
rotate
Done
rotate
Done
down
Done
down
Done
No gesture detected
rotate
Done
rotate
Done
No gesture detected
rotate
Done
down
Done
rotate
Done
rotate
Done
rotate
Done
down
Done
rotate
Done
rotate
Done
rotate
Done
rotate
Done
rotate
Done
rotate
Done
rotate
Done
rotate
Done
rotate
Done
rotate
Done
rotate
Done
rotate
Done
rotate
Done
rotate
Done
rotate
Done
rotate
Done
No gesture detected
rotate
Done
rotate
Done
No gesture detected
No gesture detected
rotate
D